In [1]:
import numpy as np
import ctypes

#Définition d'un line world sous la forme d'un MDP

In [2]:
S = [0, 1, 2 ,3, 4]
A = [0, 1] # left right
R = [-1, 0, 1]
T = [0, 4]
p = np.zeros((len(S), len(A), len(S), len(R))) # S, A, S_p, R

In [3]:
for s in range(len(S)):
  for a in range(len(A)):
    for s_p in range(len(S)):
      for r in range(len(R)):
        if s_p == s + 1 and a == 1 and R[r] == 0 and s in [1, 2]:
          p[s, a, s_p, r] = 1.0
        if s_p == s - 1 and a == 0 and R[r] == 0 and s in [2, 3]:
          p[s, a, s_p, r] = 1.0
p[3, 1, 4, 2] = 1.0
p[1, 0, 0, 0] = 1.0

# Exercice

Implémentez une fonction prenant en paramètre une policy (un tableau numpy)
et renvoyant la value de cette policy (un tableau numpy) pour chaque état du line world

In [4]:
def policy_evaluation(policy : np.ndarray, V: np.ndarray = None) -> np.ndarray:
  theta = 0.00001
  V = np.random.random(len(S))
  for s in T:
    V[s] = 0.0
  gamma = 0.999
  while True:
    delta = 0
    for s in range(len(S)):
      v = V[s]
      total = 0
      for a in range(len(A)):
        pi_s_a = policy[s, a]
        for s_p in range(len(S)):
          for r in range(len(R)):
            total += pi_s_a * p[s, a, s_p, r] * (R[r] + 0.999 * V[s_p])
      V[s] = total
      delta = np.maximum(delta, np.abs(v - V[s]))
    if delta < theta:
      return V

Définissez une policy jouant tout le temps à droite.
Affichez la value de cette policy obtenue grâce à l'algorithme policy evaluation

In [5]:
pi_right = np.array([[0.0, 1.0] for _ in range(len(S))])
print(policy_evaluation(pi_right))

[0.       0.998001 0.999    1.       0.      ]


Définissez une policy jouant tout le temps à gauche. Affichez la value de cette policy obtenue grâce à l'algorithme policy evaluation

In [6]:
pi_left = np.array([[1.0, 0.0] for _ in range(len(S))])
print(policy_evaluation(pi_left))

[ 0.       -1.       -0.999    -0.998001  0.      ]


Définissez une policy uniformément aléatoire (50% de chance d'aller à gauche et 50% de chances d'aller à droite). Affichez la value de cette policy obtenue grâce à l'algorithme policy evaluation

In [7]:
pi_random = np.array([[0.5, 0.5] for _ in range(len(S))])
print(policy_evaluation(pi_random))

[ 0.00000000e+00 -4.99993115e-01  6.87814488e-06  5.00003436e-01
  0.00000000e+00]


Définissez une policy uniformément aléatoire (15% de chance d'aller à gauche et 85% de chances d'aller à droite). Affichez la value de cette policy obtenue grâce à l'algorithme policy evaluation

In [8]:
pi_weird_random = np.array([[0.15, 0.85] for _ in range(len(S))])
print(policy_evaluation(pi_weird_random))

[0.         0.6465129  0.93801492 0.99056154 0.        ]


In [9]:
def naive_ineficient_policy_iteration():
  theta = 0.000001
  gamma = 0.999

  V = np.random.random(len(S))
  for s in T:
    V[s] = 0.0

  Pi = np.random.choice(A, len(S))

  while True:
    # Policy Evaluation
    while True:
      delta = 0.0

      for s in S:
        v = V[s]

        total = 0.0
        for s_p in S:
          for r_index in range(len(R)):
            total += p[s, Pi[s], s_p, r_index] * (R[r_index] + gamma * V[s_p])

        V[s] = total

        delta = max(delta, abs(v - V[s]))

      if delta < theta:
        break

    # Policy Improvement
    policy_stable = True

    for s in S:
      old_action = Pi[s]

      best_a = None
      best_action_score = -9999999

      for a in A:
        total = 0.0
        for s_p in S:
          for r_index in range(len(R)):
            total += p[s, a, s_p, r_index] * (R[r_index] + gamma * V[s_p])

        if best_a is None or total >= best_action_score:
          best_a = a
          best_action_score = total

      Pi[s] = best_a
      if Pi[s] != old_action:
        policy_stable = False

    if policy_stable:
      return Pi

In [10]:
naive_ineficient_policy_iteration()

array([1, 1, 1, 1, 1])

# Définissez une variante d'un GRID WORLD sous la forme d'un MDP et évaluez différentes stratégies sur cette environnement.

Le grid world est une grille de 5x5 cases (5 lignes de 5 colonnes) sur laquelle l'agent peut évoluer, il commence généralement sur la première ligne, première colonne. L'agent possède 4 actions possibles (gauche, droite, haut, bas). Si jamais l'agent atteint la dernière case de la première ligne => état terminal avec reward de -3. Si jamais l'agent atteint la dernière case de la dernière ligne => état terminal avec reward de 1. Si l'agent essaye de se déplacer en dehors des bords de la grille => état terminal avec score de -1.

In [11]:
S = np.zeros((5, 5)) 


A = [(0, -1), (0, 1), (-1, 0), (1,0)] # left right up down
R = np.zeros((5, 5))
R[0, 4] = -3
R[4, 4] = 1


p = np.zeros((S.shape[0]*S.shape[1], len(A), S.shape[0]*S.shape[1], R.shape[0] * R.shape[1])) # S, A, S_p, R

# Fonction qui calcule l'état suivant et la récompense
def transition(state, action):
    new_state = (state[0] + action[0], state[1] + action[1])
    if new_state[0] < 0 or new_state[0] >= 5 or new_state[1] < 0 or new_state[1] >= 5:
        return None, -1  # Retourne None pour indiquer un état terminal, et une récompense de -1
    else:
        return new_state, R[new_state[0], new_state[1]]

# Exemple d'utilisation de la fonction
current_state = (0, 0)  
action = (1, 0)         

# Appliquer la transition
next_state, reward = transition(current_state, action)
print("Prochain état:", next_state, "Récompense:", reward)

# Tester un déplacement hors de la grille
test_state = (0, 0)
test_action = (-1, 0)  
next_state, reward = transition(test_state, test_action)
print("Prochain état:", next_state, "Récompense:", reward)


Prochain état: (1, 0) Récompense: 0.0
Prochain état: None Récompense: -1


In [12]:
def policy_evaluation2(policy, grid_size, R, A,):
    # Initialisation de la matrice des valeurs
    V = np.zeros(grid_size)
    discount_factor=0.999
    theta=0.00001
    # Boucle jusqu'à convergence
    while True:
        delta = 0
        for i in range(grid_size[0]):
            for j in range(grid_size[1]):
                v = V[i, j]
                new_v = 0
                action = A[policy[i, j]]  # Action déterminée par la politique
                next_state, reward = transition((i, j), action)
                if next_state is None:
                    new_v = reward  # Si l'état suivant est terminal
                else:
                    new_v = reward + discount_factor * V[next_state[0], next_state[1]]
                
                V[i, j] = new_v
                delta = max(delta, abs(v - V[i, j]))
        
        # Condition de sortie de la boucle
        if delta < theta:
            break
    
    return V

In [13]:
import numpy as np
grid_size = (5, 5)
pi_left = np.zeros((5, 5), dtype=int)
print(policy_evaluation2(pi_left, S.shape, R, A))

pi_right = np.ones(grid_size, dtype=int) * 1  # Fill the matrix with 1s
print(policy_evaluation2(pi_right, S.shape, R, A))
# Policy that always moves up
pi_up = np.ones(grid_size, dtype=int) * 2     # Fill the matrix with 2s
print(policy_evaluation2(pi_up, S.shape, R, A))
# Policy that always moves down
pi_down = np.ones(grid_size, dtype=int) * 3   # Fill the matrix with 3s
print(policy_evaluation2(pi_down, S.shape, R, A))




[[-1.       -0.999    -0.998001 -0.997003 -0.996006]
 [-1.       -0.999    -0.998001 -0.997003 -0.996006]
 [-1.       -0.999    -0.998001 -0.997003 -0.996006]
 [-1.       -0.999    -0.998001 -0.997003 -0.996006]
 [-1.       -0.999    -0.998001 -0.997003 -0.996006]]
[[-3.98701499e+00 -3.99100600e+00 -3.99500100e+00 -3.99900000e+00
  -1.00000000e+00]
 [-9.96005996e-01 -9.97002999e-01 -9.98001000e-01 -9.99000000e-01
  -1.00000000e+00]
 [-9.96005996e-01 -9.97002999e-01 -9.98001000e-01 -9.99000000e-01
  -1.00000000e+00]
 [-9.96005996e-01 -9.97002999e-01 -9.98001000e-01 -9.99000000e-01
  -1.00000000e+00]
 [ 9.97002999e-04  9.98001000e-04  9.99000000e-04  1.00000000e-03
  -1.00000000e+00]]
[[-1.         -1.         -1.         -1.         -1.        ]
 [-0.999      -0.999      -0.999      -0.999      -3.999     ]
 [-0.998001   -0.998001   -0.998001   -0.998001   -3.995001  ]
 [-0.997003   -0.997003   -0.997003   -0.997003   -3.991006  ]
 [-0.996006   -0.996006   -0.996006   -0.996006   -3.987

# Proposez plusieurs stratégies et évaluez les à l'aide de policy évaluation.

In [18]:
pi_right = np.array([[0.0, 1.0] for _ in range(len(S))])
policy_evaluation(pi_right)

IndexError: index 2 is out of bounds for axis 1 with size 2

# Bonus : Implémentez policy itération et value itération (dans les slides que nous n'avons pas encore vu) et obtenez à l'aide de ces derniers pi* pour le Line World et le Grid World

In [15]:
#TODO

# Naive Policy iteration implementation in order to find a Pi*

In [16]:
def naive_policy_iteration():
  theta = 0.00001
  gamma = 0.999

  V = np.random.random(len(S))
  for s in T:
    V[s] = 0.0
  Pi = np.random.choice(A, len(S), True)

  while True:
    # Policy evaluation
    while True:
      delta = 0
      for s in range(len(S)):
        v = V[s]
        total = 0
        for s_p in range(len(S)):
          for r in range(len(R)):
            total += p[s, Pi[s], s_p, r] * (R[r] + 0.999 * V[s_p])
        V[s] = total
        delta = np.maximum(delta, np.abs(v - V[s]))
      if delta < theta:
        break

    # Policy improvement
    policy_stable = True

    for s in S:
      if s in T:
        continue

      old_action = Pi[s]

      # Compute Arg Max a
      argmax_a = None
      max_a = -999999999

      for a in A:
        total = 0.0

        for s_p in S:
          for r_index in range(len(R)):
            total += p[s, a, s_p, r_index] * (R[r_index] + gamma * V[s_p])

        if argmax_a is None or total >= max_a:
          argmax_a = a
          max_a = total

      Pi[s] = argmax_a

      if old_action != Pi[s]:
        policy_stable = False


    if policy_stable:
      break

  return Pi


In [17]:
naive_policy_iteration()

ValueError: a must be 1-dimensional